In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import load_model, Sequential
import tensorflow as tf
from art.estimators.classification import KerasClassifier
from art.attacks.extraction import CopycatCNN
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

tf.compat.v1.disable_eager_execution()

DIMENSION = 256
loaded_model = load_model('./target_model')
x_stealed_loaded = np.load('x_val.npy')
y_stealed_loaded = np.loadtxt('y_val.txt', delimiter=',')

c:\Users\andre\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(x_stealed_loaded, y_stealed_loaded, test_size=0.30, random_state=1)

# Ataque de extraccion

In [3]:
classifier = KerasClassifier(loaded_model)

In [4]:
copycat_cnn = CopycatCNN(
  batch_size_fit=16,
  batch_size_query=16,
  nb_epochs=5,
  nb_stolen=500,
  classifier=classifier
)

In [5]:
stolen_m = Sequential()
stolen_m.add(Convolution2D(32, (3, 3), strides=(1, 1), input_shape=(DIMENSION, DIMENSION, 3), activation='relu'))
stolen_m.add(MaxPooling2D(pool_size=(2, 2)))
stolen_m.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
stolen_m.add(MaxPooling2D(pool_size=(2, 2)))
stolen_m.add(Flatten())
stolen_m.add(Dense(512, activation='softplus'))
stolen_m.add(Dense(25, activation='sigmoid'))

stolen_m.compile(
  optimizer="adam",
  loss="categorical_crossentropy",
  metrics=["accuracy"]
)

In [6]:
model_stolen = KerasClassifier(model=stolen_m)

In [7]:
stolen_classifier = copycat_cnn.extract(
  x=X_train_s, 
  y=y_train_s, 
  thieved_classifier=model_stolen
)

c:\Users\andre\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 500 samples
Epoch 1/5
500/500 [==============================] - 48s 96ms/sample - loss: 3788.7060 - accuracy: 0.3440
Epoch 2/5
500/500 [==============================] - 38s 75ms/sample - loss: 1.1928 - accuracy: 0.7280
Epoch 3/5
500/500 [==============================] - 37s 75ms/sample - loss: 0.2941 - accuracy: 0.9380
Epoch 4/5
500/500 [==============================] - 37s 75ms/sample - loss: 0.1926 - accuracy: 0.9480
Epoch 5/5
500/500 [==============================] - 37s 75ms/sample - loss: 0.0848 - accuracy: 0.9680


In [8]:
score_original = classifier._model.evaluate(
  x=X_test_s,
  y=y_test_s
)

score_stolen = stolen_classifier._model.evaluate(
  x=X_test_s,
  y=y_test_s
)

# Comparing test losses
print(f"Original test loss: {score_original[0]:.2f} "  f"vs stolen test loss: {score_stolen[0]:.2f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.2f} " f"vs stolen test accuracy: {score_stolen[1]:.2f}")

c:\Users\andre\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Original test loss: 0.76 vs stolen test loss: 6.96
Original test accuracy: 0.90 vs stolen test accuracy: 0.60
